In [352]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split 
torch.cuda.is_available()

True

In [354]:
path_v6 = '/root/COLM/dataset/mmlu_unified_data_v6.csv'
data = pd.read_csv(path_v6, encoding="utf-8")
print(data.shape)
data.head()

(15857, 16)


,uid,question,gold_answer,split,subject,auto_answers_gemma-7b-it,auto_answers_llama2-13b-chat,auto_answers_mistral-7b-inst,gold_answer_from_lmdf,auto_answers_falcon-7b,auto_answers_gemma-7b,auto_answers_llama2-7b-lm,auto_answers_metamath-7b,auto_answers_mistral-7b-lm,auto_answers_olmo-7b,auto_answers_opt-6.7b
0,train-abstract_algebra-0,Find all c in Z_3 such that Z_3[x]/(x^2 + c) i...,B,dev,abstract_algebra,"['A', 'A', 'A', 'A', 'A', 'B', 'A', 'C', 'A', ...","['D', 'D', 'A', 'INVALID', 'D', 'D', 'A', 'D',...","['A', 'INVALID', 'B', 'C', 'A', 'B', 'D', 'B',...",B,"['B', 'B', 'B', 'B', 'C', 'B', 'B', 'B', 'B', ...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...","['B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', ...","['B', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'A', ..."
1,train-abstract_algebra-1,Statement 1 | If aH is an element of a factor ...,B,dev,abstract_algebra,"['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'C', 'C', 'A', 'D', 'D', 'D', 'D', 'A', ...","['D', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...",B,"['C', 'A', 'A', 'B', 'B', 'B', 'B', 'A', 'B', ...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...","['B', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', ...","['B', 'B', 'B', 'INVALID', 'B', 'B', 'B', 'B',...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...","['B', 'D', 'B', 'B', 'D', 'A', 'B', 'C', 'B', ...","['A', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'A', ..."
2,train-abstract_algebra-2,Statement 1 | Every element of a group generat...,C,dev,abstract_algebra,"['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['C', 'D', 'A', 'A', 'A', 'A', 'B', 'C', 'A', ...","['C', 'A', 'D', 'C', 'A', 'B', 'A', 'B', 'C', ...",C,"['B', 'C', 'B', 'A', 'B', 'A', 'B', 'D', 'B', ...","['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...","['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...","['B', 'B', 'B', 'C', 'D', 'INVALID', 'B', 'B',...","['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...","['C', 'A', 'A', 'C', 'C', 'C', 'A', 'C', 'D', ...","['A', 'A', 'A', 'D', 'A', 'A', 'A', 'A', 'A', ..."
3,train-abstract_algebra-3,Statement 1| Every function from a finite set ...,A,dev,abstract_algebra,"['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['C', 'A', 'A', 'C', 'C', 'A', 'C', 'C', 'C', ...","['C', 'INVALID', 'A', 'D', 'B', 'A', 'A', 'A',...",A,"['INVALID', 'B', 'A', 'A', 'B', 'C', 'A', 'INV...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'A', 'A', 'D', 'A', 'B', 'A', 'A', 'A', ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ..."
4,train-abstract_algebra-4,Find the characteristic of the ring 2Z. (A) 0 ...,A,dev,abstract_algebra,"['A', 'A', 'A', 'B', 'C', 'C', 'A', 'B', 'A', ...","['A', 'D', 'B', 'B', 'C', 'D', 'C', 'A', 'B', ...","['A', 'A', 'A', 'A', 'A', 'A', 'INVALID', 'A',...",A,"['A', 'C', 'C', 'INVALID', 'D', 'A', 'C', 'D',...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'INVALID', 'INVALID', 'INVALID', 'A', 'I...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'A', 'C', 'A', 'A', 'C', 'A', 'A', 'A', ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ..."


In [355]:
retain_cols = ['uid',
 'question',
 'gold_answer',
 'split',
 'subject',
 'auto_answers_gemma-7b-it',
 'auto_answers_llama2-13b-chat',
 'auto_answers_mistral-7b-inst',
 'auto_answers_gemma-7b',
 'auto_answers_llama2-7b-lm',
 'auto_answers_metamath-7b',
 'auto_answers_mistral-7b-lm']

data_all = data[retain_cols]

column_names = data_all.columns.tolist()
print(column_names)

['uid', 'question', 'gold_answer', 'split', 'subject', 'auto_answers_gemma-7b-it', 'auto_answers_llama2-13b-chat', 'auto_answers_mistral-7b-inst', 'auto_answers_gemma-7b', 'auto_answers_llama2-7b-lm', 'auto_answers_metamath-7b', 'auto_answers_mistral-7b-lm']


In [356]:
data_all.rename(columns = {'auto_answers_gemma-7b-it':'auto_answers_maj_gemma-7b-it_CHAT', 'auto_answers_llama2-13b-chat':'auto_answers_maj_llama2-13b-chat_CHAT',\
                           'auto_answers_mistral-7b-inst':'auto_answers_maj_mistral-7b-inst_CHAT','auto_answers_gemma-7b':'auto_answers_maj_gemma-7b_LM',\
                         'auto_answers_llama2-7b-lm':'auto_answers_maj_llama2-7b-lm_LM', 'auto_answers_metamath-7b':'auto_answers_maj_metamath-7b_LM',\
                         'auto_answers_mistral-7b-lm':'auto_answers_maj_mistral-7b-lm_LM'}, inplace = True)

print(data_all.shape)
data_all.head()

(15857, 12)


/tmp/ipykernel_4367/3924950439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all.rename(columns = {'auto_answers_gemma-7b-it':'auto_answers_maj_gemma-7b-it_CHAT', 'auto_answers_llama2-13b-chat':'auto_answers_maj_llama2-13b-chat_CHAT',\


,uid,question,gold_answer,split,subject,auto_answers_maj_gemma-7b-it_CHAT,auto_answers_maj_llama2-13b-chat_CHAT,auto_answers_maj_mistral-7b-inst_CHAT,auto_answers_maj_gemma-7b_LM,auto_answers_maj_llama2-7b-lm_LM,auto_answers_maj_metamath-7b_LM,auto_answers_maj_mistral-7b-lm_LM
0,train-abstract_algebra-0,Find all c in Z_3 such that Z_3[x]/(x^2 + c) i...,B,dev,abstract_algebra,"['A', 'A', 'A', 'A', 'A', 'B', 'A', 'C', 'A', ...","['D', 'D', 'A', 'INVALID', 'D', 'D', 'A', 'D',...","['A', 'INVALID', 'B', 'C', 'A', 'B', 'D', 'B',...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ..."
1,train-abstract_algebra-1,Statement 1 | If aH is an element of a factor ...,B,dev,abstract_algebra,"['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'C', 'C', 'A', 'D', 'D', 'D', 'D', 'A', ...","['D', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...","['B', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', ...","['B', 'B', 'B', 'INVALID', 'B', 'B', 'B', 'B',...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ..."
2,train-abstract_algebra-2,Statement 1 | Every element of a group generat...,C,dev,abstract_algebra,"['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['C', 'D', 'A', 'A', 'A', 'A', 'B', 'C', 'A', ...","['C', 'A', 'D', 'C', 'A', 'B', 'A', 'B', 'C', ...","['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...","['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...","['B', 'B', 'B', 'C', 'D', 'INVALID', 'B', 'B',...","['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ..."
3,train-abstract_algebra-3,Statement 1| Every function from a finite set ...,A,dev,abstract_algebra,"['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['C', 'A', 'A', 'C', 'C', 'A', 'C', 'C', 'C', ...","['C', 'INVALID', 'A', 'D', 'B', 'A', 'A', 'A',...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ..."
4,train-abstract_algebra-4,Find the characteristic of the ring 2Z. (A) 0 ...,A,dev,abstract_algebra,"['A', 'A', 'A', 'B', 'C', 'C', 'A', 'B', 'A', ...","['A', 'D', 'B', 'B', 'C', 'D', 'C', 'A', 'B', ...","['A', 'A', 'A', 'A', 'A', 'A', 'INVALID', 'A',...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'INVALID', 'INVALID', 'INVALID', 'A', 'I...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ..."


In [357]:
def mostfreqentval(mylist):
    my_list = [ item.strip() for item in mylist]
    
    counter = Counter(my_list)

    # Find the most common elements and their counts
    most_common_values = counter.most_common()
    if len(most_common_values) == 0:
        return 0.9999989898999998989899
    else:
        return most_common_values[0][0]


def llm_performance(ans, model):
    assert len(ans) == len(model)

    count = 0
    for i, (a, m) in enumerate(zip(ans, model)):
        fval = mostfreqentval(eval(m))
        if a.strip() == fval.strip():
            count +=1

    print("Performance :", (count/len(ans))*100)
        
    
    
llm_performance(data_all['gold_answer'].values.tolist(), data_all['auto_answers_maj_gemma-7b_LM'].values.tolist())
llm_performance(data_all['gold_answer'].values.tolist(), data_all['auto_answers_maj_llama2-7b-lm_LM'].values.tolist())
llm_performance(data_all['gold_answer'].values.tolist(), data_all['auto_answers_maj_metamath-7b_LM'].values.tolist())
llm_performance(data_all['gold_answer'].values.tolist(), data_all['auto_answers_maj_mistral-7b-lm_LM'].values.tolist())
llm_performance(data_all['gold_answer'].values.tolist(), data_all['auto_answers_maj_gemma-7b-it_CHAT'].values.tolist())
llm_performance(data_all['gold_answer'].values.tolist(), data_all['auto_answers_maj_llama2-13b-chat_CHAT'].values.tolist())
llm_performance(data_all['gold_answer'].values.tolist(), data_all['auto_answers_maj_mistral-7b-inst_CHAT'].values.tolist())

Performance : 66.44384183641294
Performance : 47.87160244686889
Performance : 42.883269218641615
Performance : 62.33839944504005
Performance : 50.57072586239516
Performance : 54.247335561581636
Performance : 53.477959260894245


In [358]:
def mostfreqentval(mylist, gold_val):
    my_list = [ item.strip() for item in mylist]
    counter = Counter(my_list)
    most_common_values = counter.most_common()
    if len(most_common_values) == 0:
        topfrq = 0.9999989898999998989899
    else:
        topfrq = most_common_values[0][0]
    if topfrq.strip() == gold_val.strip():
        return 1
    else:
        return 0

kepping_llms = ["maj_gemma-7b-it_CHAT", "maj_llama2-13b-chat_CHAT", "maj_mistral-7b-inst_CHAT", "maj_mistral-7b-lm_LM", "maj_gemma-7b_LM", "maj_metamath-7b_LM", "maj_llama2-7b-lm_LM"]
 
for llm_name in kepping_llms:
    data_all[llm_name] = [ mostfreqentval(eval(item), gval) for item, gval in zip(data_all["auto_answers_"+str(llm_name)].values.tolist(), data_all["gold_answer"].values.tolist())]

data_all.head()

/tmp/ipykernel_4367/1354196263.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all[llm_name] = [ mostfreqentval(eval(item), gval) for item, gval in zip(data_all["auto_answers_"+str(llm_name)].values.tolist(), data_all["gold_answer"].values.tolist())]
/tmp/ipykernel_4367/1354196263.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all[llm_name] = [ mostfreqentval(eval(item), gval) for item, gval in zip(data_all["auto_answers_"+str(llm_name)].values.tolist(), data_all["gold_answer"].values.t

,uid,question,gold_answer,split,subject,auto_answers_maj_gemma-7b-it_CHAT,auto_answers_maj_llama2-13b-chat_CHAT,auto_answers_maj_mistral-7b-inst_CHAT,auto_answers_maj_gemma-7b_LM,auto_answers_maj_llama2-7b-lm_LM,auto_answers_maj_metamath-7b_LM,auto_answers_maj_mistral-7b-lm_LM,maj_gemma-7b-it_CHAT,maj_llama2-13b-chat_CHAT,maj_mistral-7b-inst_CHAT,maj_mistral-7b-lm_LM,maj_gemma-7b_LM,maj_metamath-7b_LM,maj_llama2-7b-lm_LM
0,train-abstract_algebra-0,Find all c in Z_3 such that Z_3[x]/(x^2 + c) i...,B,dev,abstract_algebra,"['A', 'A', 'A', 'A', 'A', 'B', 'A', 'C', 'A', ...","['D', 'D', 'A', 'INVALID', 'D', 'D', 'A', 'D',...","['A', 'INVALID', 'B', 'C', 'A', 'B', 'D', 'B',...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...",0,0,1,1,1,1,1
1,train-abstract_algebra-1,Statement 1 | If aH is an element of a factor ...,B,dev,abstract_algebra,"['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'C', 'C', 'A', 'D', 'D', 'D', 'D', 'A', ...","['D', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...","['B', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', ...","['B', 'B', 'B', 'INVALID', 'B', 'B', 'B', 'B',...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...",0,0,0,1,1,1,1
2,train-abstract_algebra-2,Statement 1 | Every element of a group generat...,C,dev,abstract_algebra,"['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['C', 'D', 'A', 'A', 'A', 'A', 'B', 'C', 'A', ...","['C', 'A', 'D', 'C', 'A', 'B', 'A', 'B', 'C', ...","['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...","['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...","['B', 'B', 'B', 'C', 'D', 'INVALID', 'B', 'B',...","['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,0,0,1,1,0,1
3,train-abstract_algebra-3,Statement 1| Every function from a finite set ...,A,dev,abstract_algebra,"['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['C', 'A', 'A', 'C', 'C', 'A', 'C', 'C', 'C', ...","['C', 'INVALID', 'A', 'D', 'B', 'A', 'A', 'A',...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...",1,0,1,1,1,1,1
4,train-abstract_algebra-4,Find the characteristic of the ring 2Z. (A) 0 ...,A,dev,abstract_algebra,"['A', 'A', 'A', 'B', 'C', 'C', 'A', 'B', 'A', ...","['A', 'D', 'B', 'B', 'C', 'D', 'C', 'A', 'B', ...","['A', 'A', 'A', 'A', 'A', 'A', 'INVALID', 'A',...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'INVALID', 'INVALID', 'INVALID', 'A', 'I...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...",1,0,1,1,1,0,1


In [359]:
#best GSM8K Models
# maj_metamath-7b_LM: 6272 times (92.02%)
# maj_gemma-7b_LM: 5025 times (73.72%)
# maj_mistral-7b-lm_LM: 4023 times (59.02%)
# maj_mistral-7b-inst_CHAT: 3419 times (50.16%)
# maj_llama2-13b-chat_CHAT: 3311 times (48.58%)
# maj_gemma-7b-it_CHAT: 2386 times (35.01%)

#Best MMLU Models:
# maj_gemma-7b_LM: 9114 times (66.25%)
# maj_mistral-7b-lm_LM: 8521 times (61.94%)
# maj_llama2-13b-chat_CHAT: 7188 times (52.25%)
# maj_mistral-7b-inst_CHAT: 7002 times (50.90%)
# maj_gemma-7b-it_CHAT: 6787 times (49.33%)
# maj_llama2-7b-lm_LM: 6421 times (46.67%)
# maj_metamath-7b_LM: 5813 times (42.25%)


keep_llms = ["maj_metamath-7b_LM", "maj_gemma-7b-it_CHAT", "maj_mistral-7b-inst_CHAT", "maj_llama2-7b-lm_LM", "maj_llama2-13b-chat_CHAT", "maj_mistral-7b-lm_LM", "maj_gemma-7b_LM"]

# print(keep_llms_old)
# keep_llms = keep_llms_old[::-1]
# print(keep_llms)
    
for col_name in keep_llms:
    data_all[col_name] = [ int(item) for item in data_all[col_name].values.tolist()]

for i in range(len(keep_llms)-1):
    subset_llms =  keep_llms[0:i+2]
    id2mod = { i:val for i, val in enumerate(subset_llms)} 
    mod2id = { val:1 for i, val in enumerate(subset_llms)}
    print(subset_llms)
    data_all[f'maj_{i+2}'] = [ [ int(val) for val in item ] for item in data_all[subset_llms].values.tolist()]
    data_all[f'maj_{i+2}_model'] = [ [ id2mod[i] for i, val in enumerate(item) if id2mod[i] in subset_llms and val != 0] for item in data_all[f'maj_{i+2}'].tolist()]

data_all.head()

['maj_metamath-7b_LM', 'maj_gemma-7b-it_CHAT']
['maj_metamath-7b_LM', 'maj_gemma-7b-it_CHAT', 'maj_mistral-7b-inst_CHAT']
['maj_metamath-7b_LM', 'maj_gemma-7b-it_CHAT', 'maj_mistral-7b-inst_CHAT', 'maj_llama2-7b-lm_LM']
['maj_metamath-7b_LM', 'maj_gemma-7b-it_CHAT', 'maj_mistral-7b-inst_CHAT', 'maj_llama2-7b-lm_LM', 'maj_llama2-13b-chat_CHAT']
['maj_metamath-7b_LM', 'maj_gemma-7b-it_CHAT', 'maj_mistral-7b-inst_CHAT', 'maj_llama2-7b-lm_LM', 'maj_llama2-13b-chat_CHAT', 'maj_mistral-7b-lm_LM']
['maj_metamath-7b_LM', 'maj_gemma-7b-it_CHAT', 'maj_mistral-7b-inst_CHAT', 'maj_llama2-7b-lm_LM', 'maj_llama2-13b-chat_CHAT', 'maj_mistral-7b-lm_LM', 'maj_gemma-7b_LM']


,uid,question,gold_answer,split,subject,auto_answers_maj_gemma-7b-it_CHAT,auto_answers_maj_llama2-13b-chat_CHAT,auto_answers_maj_mistral-7b-inst_CHAT,auto_answers_maj_gemma-7b_LM,auto_answers_maj_llama2-7b-lm_LM,...,maj_3,maj_3_model,maj_4,maj_4_model,maj_5,maj_5_model,maj_6,maj_6_model,maj_7,maj_7_model
0,train-abstract_algebra-0,Find all c in Z_3 such that Z_3[x]/(x^2 + c) i...,B,dev,abstract_algebra,"['A', 'A', 'A', 'A', 'A', 'B', 'A', 'C', 'A', ...","['D', 'D', 'A', 'INVALID', 'D', 'D', 'A', 'D',...","['A', 'INVALID', 'B', 'C', 'A', 'B', 'D', 'B',...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...",...,"[1, 0, 1]","[maj_metamath-7b_LM, maj_mistral-7b-inst_CHAT]","[1, 0, 1, 1]","[maj_metamath-7b_LM, maj_mistral-7b-inst_CHAT,...","[1, 0, 1, 1, 0]","[maj_metamath-7b_LM, maj_mistral-7b-inst_CHAT,...","[1, 0, 1, 1, 0, 1]","[maj_metamath-7b_LM, maj_mistral-7b-inst_CHAT,...","[1, 0, 1, 1, 0, 1, 1]","[maj_metamath-7b_LM, maj_mistral-7b-inst_CHAT,..."
1,train-abstract_algebra-1,Statement 1 | If aH is an element of a factor ...,B,dev,abstract_algebra,"['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'C', 'C', 'A', 'D', 'D', 'D', 'D', 'A', ...","['D', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', ...","['B', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', ...",...,"[1, 0, 0]",[maj_metamath-7b_LM],"[1, 0, 0, 1]","[maj_metamath-7b_LM, maj_llama2-7b-lm_LM]","[1, 0, 0, 1, 0]","[maj_metamath-7b_LM, maj_llama2-7b-lm_LM]","[1, 0, 0, 1, 0, 1]","[maj_metamath-7b_LM, maj_llama2-7b-lm_LM, maj_...","[1, 0, 0, 1, 0, 1, 1]","[maj_metamath-7b_LM, maj_llama2-7b-lm_LM, maj_..."
2,train-abstract_algebra-2,Statement 1 | Every element of a group generat...,C,dev,abstract_algebra,"['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['C', 'D', 'A', 'A', 'A', 'A', 'B', 'C', 'A', ...","['C', 'A', 'D', 'C', 'A', 'B', 'A', 'B', 'C', ...","['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...","['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",...,"[0, 0, 0]",[],"[0, 0, 0, 1]",[maj_llama2-7b-lm_LM],"[0, 0, 0, 1, 0]",[maj_llama2-7b-lm_LM],"[0, 0, 0, 1, 0, 1]","[maj_llama2-7b-lm_LM, maj_mistral-7b-lm_LM]","[0, 0, 0, 1, 0, 1, 1]","[maj_llama2-7b-lm_LM, maj_mistral-7b-lm_LM, ma..."
3,train-abstract_algebra-3,Statement 1| Every function from a finite set ...,A,dev,abstract_algebra,"['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['C', 'A', 'A', 'C', 'C', 'A', 'C', 'C', 'C', ...","['C', 'INVALID', 'A', 'D', 'B', 'A', 'A', 'A',...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...",...,"[1, 1, 1]","[maj_metamath-7b_LM, maj_gemma-7b-it_CHAT, maj...","[1, 1, 1, 1]","[maj_metamath-7b_LM, maj_gemma-7b-it_CHAT, maj...","[1, 1, 1, 1, 0]","[maj_metamath-7b_LM, maj_gemma-7b-it_CHAT, maj...","[1, 1, 1, 1, 0, 1]","[maj_metamath-7b_LM, maj_gemma-7b-it_CHAT, maj...","[1, 1, 1, 1, 0, 1, 1]","[maj_metamath-7b_LM, maj_gemma-7b-it_CHAT, maj..."
4,train-abstract_algebra-4,Find the characteristic of the ring 2Z. (A) 0 ...,A,dev,abstract_algebra,"['A', 'A', 'A', 'B', 'C', 'C', 'A', 'B', 'A', ...","['A', 'D', 'B', 'B', 'C', 'D', 'C', 'A', 'B', ...","['A', 'A', 'A', 'A', 'A', 'A', 'INVALID', 'A',...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...","['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', ...",...,"[0, 1, 1]","[maj_gemma-7b-it_CHAT, maj_mistral-7b-inst_CHAT]","[0, 1, 1, 1]","[maj_gemma-7b-it_CHAT, maj_mistral-7b-inst_CHA...","[0, 1, 1, 1, 0]","[maj_gemma-7b-it_CHAT, maj_mistral-7b-inst_CHA...","[0, 1, 1, 1, 0, 1]","[maj_gemma-7b-it_CHAT, maj_mistral-7b-inst_CHA...","[0, 1, 1, 1, 0, 1, 1]","[maj_gemma-7b-it_CHAT, maj_mistral-7b-inst_CHA..."


In [360]:
map_split = {'val':'test', 'test':'train', 'dev':'val'}
data_all['split'] = [map_split[item] for item in data_all['split'].values.tolist()]

train_data = data_all.loc[data_all['split'] == 'train']
test_data = data_all.loc[data_all['split'] == 'test']
val_data = data_all.loc[data_all['split'] == 'val']
print(train_data.shape, val_data.shape, test_data.shape)

(14042, 31) (285, 31) (1530, 31)


In [361]:
#label Distribution 
from collections import Counter

def lab_dist(all_in):
    temp = [ [ keep_llms[i] for i, val in enumerate(item) if val !=0 ] for item in all_in]
    flattened_list = [item for sublist in temp for item in sublist]

    # Count the occurrences of each element
    element_counts = Counter(flattened_list)

    # Display the counts
    for element, count in element_counts.items():
        percentage = (count / len(all_in)) * 100
        print(f"{element}: {count} times ({percentage:.2f}%)")
        
print("*********** Train Data *************")
lab_dist(train_data['maj_7'].tolist())
print("")
print("*********** Validation Data ****************")
lab_dist(val_data['maj_7'].tolist())
print("")
print("*********** Test Data ****************")
lab_dist(test_data['maj_7'].tolist())
print("*********** All Data ****************")
lab_dist(data_all['maj_7'].tolist())

*********** Train Data *************
maj_metamath-7b_LM: 5920 times (42.16%)
maj_gemma-7b-it_CHAT: 7107 times (50.61%)
maj_llama2-7b-lm_LM: 6578 times (46.85%)
maj_gemma-7b_LM: 9282 times (66.10%)
maj_mistral-7b-inst_CHAT: 7497 times (53.39%)
maj_llama2-13b-chat_CHAT: 7639 times (54.40%)
maj_mistral-7b-lm_LM: 8663 times (61.69%)

*********** Validation Data ****************
maj_metamath-7b_LM: 241 times (84.56%)
maj_mistral-7b-inst_CHAT: 158 times (55.44%)
maj_llama2-7b-lm_LM: 277 times (97.19%)
maj_mistral-7b-lm_LM: 280 times (98.25%)
maj_gemma-7b_LM: 279 times (97.89%)
maj_gemma-7b-it_CHAT: 136 times (47.72%)
maj_llama2-13b-chat_CHAT: 153 times (53.68%)

*********** Test Data ****************
maj_gemma-7b-it_CHAT: 776 times (50.72%)
maj_mistral-7b-inst_CHAT: 825 times (53.92%)
maj_llama2-7b-lm_LM: 736 times (48.10%)
maj_gemma-7b_LM: 975 times (63.73%)
maj_metamath-7b_LM: 639 times (41.76%)
maj_mistral-7b-lm_LM: 942 times (61.57%)
maj_llama2-13b-chat_CHAT: 810 times (52.94%)
*********

In [362]:
print("Saving to MMLU")
train_data.to_csv('/root/COLM/dataset/mmlu/train_all_bot.csv', index=False, encoding="utf-8")
val_data.to_csv('/root/COLM/dataset/mmlu/val_all_bot.csv', index=False, encoding="utf-8")
data_all.to_csv('/root/COLM/dataset/mmlu/test_all_bot.csv', index=False, encoding="utf-8")
#data_all.to_csv('/root/COLM/dataset/mmlu/mmlu_unified_v6.csv', index=False, encoding="utf-8")

Saving to MMLU
